In [9]:
# from spotify_api import SpotifyData
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify
import yaml
import pandas as pd

def login():
    with open('config.yml', "r") as f:
        client = yaml.safe_load(f)
    client_id = client['client_id']
    client_secret= client['client_secret']
    sp = Spotify(client_credentials_manager=SpotifyClientCredentials(
        client_id = client_id,
        client_secret = client_secret
    ))
    return sp
def sc(sp, type='track,artist,album',q='the%201975'):
    req_search = sp.search(q=q,type=type)
    tracks =[]
    for item in req_search['tracks']['items']:
        track = {}
        track['track'] = item['name']
        track['release_date'] = item['album']['release_date']
        track['popularity'] = item['popularity']
        track['id'] = item['id']
        track['artist'] = item['artists'][0]['name']
        track['artist_id'] = item['artists'][0]['id']
        track['album'] = item['album']['name']
        track['album_id'] = item['album']['id']
        track['type'] = item['type']
        tracks.append(track)
    tracks = pd.DataFrame(tracks)
    return tracks


sp=login()
tracks = sc(sp,q='David Bowie')
sp.audio_analysis(tracks.loc[0,'id'])

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1571506460,
  'analysis_time': 7.62892,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 5478102,
  'duration': 248.44,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.19628,
  'start_of_fade_out': 224.73433,
  'loudness': -7.813,
  'tempo': 113.809,
  'tempo_confidence': 0.747,
  'time_signature': 4,
  'time_signature_confidence': 0.975,
  'key': 2,
  'key_confidence': 0.681,
  'mode': 1,
  'mode_confidence': 0.603,
  'codestring': 'eJxVmgeSbDcOBK_SR6A397_YZhZ79HtDIYXIeU0DUygAvOe21vaun_K55-7dW5mfuv5Ge_TPrO0zyx7t7NM_ex3_1kvbpX9qaePT5r1r9LM_dZTzWa3WeybLzL4-u67WT2GHum_zp7OsVe-nlXG-uwzWbZ2P_23Kmr-7tjN-t-21_G7bZyufw1HqmLV--uLjUWrvd_X76ZeVRyl37LvmZ3xvxxl35bv931LtrM9sdf27wdx82zonn_zzWd8jl95nr3y22_fIpTGc_ye25ZF_5Fbf_b43QNDn5wZ7j_GZs99S772ffTnU

In [2]:
from lyrics_rub import *
lr = LyricsRub()
lyrics_str, lyrics_line_lst = lr.process_lyrics(lr.get_lyrics('some%20steve%20lacy'))
print(lyrics_str)
print(lyrics_line_lst)

Steve-Lacy_Some. Baby, I want some of your love. Your love, your love, your love. Baby, can I have some of your loveYour love, your love, your love. I know there′s lots of love in your heart. But, who you gonna give it to?. Oh-oh-oh, I'm sure there′s lots guys that you see. But, I swear they are not as cool as me. Girl I can take you where you wanna go. If you wanna run, we watch a movie. Hit the beach, or just chill and get high. And we can do whatever you wanna do. Baby, I want some of your love. Your love, your love, your love. Baby, can I have some of your love. Your love, your love, your love
['Steve-Lacy_Some', 'Baby, I want some of your love', 'Your love, your love, your love', 'Baby, can I have some of your loveYour love, your love, your love', 'I know there′s lots of love in your heart', 'But, who you gonna give it to?', "Oh-oh-oh, I'm sure there′s lots guys that you see", 'But, I swear they are not as cool as me', 'Girl I can take you where you wanna go', 'If you wanna run, w